# Training a simple neural network on the MNIST Dataset with pytorch
This is the start of the journey to learn pytorch. I already know tensorflow, its just transpile (:d) my knowledge

## Importing

In [1]:
import torch, random, numpy as np
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
from torchvision import datasets, transforms

In [2]:
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # si usas multi-GPU
np.random.seed(SEED)
random.seed(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Dataset splits and preprocessing 

We make a simple preprocessing because the MNIST dataset don't require huge processing.

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),       
    transforms.Normalize((0.5,), (0.5,)) #this is like the scaler of tensorflow
])


train_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./data", train=True, download=True, transform=transform), #we use
    batch_size=64,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./data", train=False, download=True, transform=transform),
    batch_size=1000,
    shuffle=False
)

## Neural Network arquitecture

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1,16,3,1)
        self.conv2 = nn.Conv2d(16,32,3,1)
        self.full1 = nn.Linear(12*12*32,128)
        self.full2 = nn.Linear(128,10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2)
        x = torch.flatten(x,1)
        x = F.relu(self.full1(x))
        x = self.full2(x)

        return x

In [5]:
model = SimpleCNN()

In [6]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
metric = nn.CrossEntropyLoss()

## Training Function

In [7]:
def train(model, train_loader, device, metric, epochs, optimizer):
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader):

        data, targets = data.to(device), targets.to(device)

        optimizer.zero_grad()

        output = model(data)

        loss = metric(output, targets)

        loss.backward()

        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Época {epoch} | Lote {batch_idx} | Pérdida: {loss.item():.4f}")

## Evaluating function

In [8]:
def evaluate(model, device, test_loader, metric):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for batch_idx, (data, targets) in enumerate(test_loader):
            data, targets = data.to(device), targets.to(device)

            output = model(data)

            test_loss += metric(output, targets).item()

            pred = output.argmax(dim=1, keepdim=True)

            correct += pred.eq(targets.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        accuracy = 100 * correct / len(test_loader.dataset)

    print(f"\nPérdida promedio en test: {test_loss:.4f}, Precisión: {accuracy:.2f}%\n")
    return test_loss, accuracy
    

## Training and evaluation

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for epoch in range(1, 6):  # entrenamos 5 épocas
    train(model, train_loader, device, metric, epoch, optimizer)
    evaluate(model, device, test_loader, metric)


Época 1 | Lote 0 | Pérdida: 2.2965
Época 1 | Lote 100 | Pérdida: 0.2903
Época 1 | Lote 200 | Pérdida: 0.1690
Época 1 | Lote 300 | Pérdida: 0.0579
Época 1 | Lote 400 | Pérdida: 0.0512
Época 1 | Lote 500 | Pérdida: 0.1974
Época 1 | Lote 600 | Pérdida: 0.0400
Época 1 | Lote 700 | Pérdida: 0.0152
Época 1 | Lote 800 | Pérdida: 0.1052
Época 1 | Lote 900 | Pérdida: 0.1103

Pérdida promedio en test: 0.0001, Precisión: 98.09%

Época 2 | Lote 0 | Pérdida: 0.1785
Época 2 | Lote 100 | Pérdida: 0.0392
Época 2 | Lote 200 | Pérdida: 0.0049
Época 2 | Lote 300 | Pérdida: 0.0515
Época 2 | Lote 400 | Pérdida: 0.0363
Época 2 | Lote 500 | Pérdida: 0.0446
Época 2 | Lote 600 | Pérdida: 0.1856
Época 2 | Lote 700 | Pérdida: 0.1785
Época 2 | Lote 800 | Pérdida: 0.0106
Época 2 | Lote 900 | Pérdida: 0.0549

Pérdida promedio en test: 0.0000, Precisión: 98.47%

Época 3 | Lote 0 | Pérdida: 0.0124
Época 3 | Lote 100 | Pérdida: 0.0560
Época 3 | Lote 200 | Pérdida: 0.0027
Época 3 | Lote 300 | Pérdida: 0.0193
Época 3 | 

In [10]:
# Guardar pesos
torch.save(model.state_dict(), "modelo_mnist.pth")

# Cargar pesos después
model = SimpleCNN()              # recreas el modelo
model.load_state_dict(torch.load("modelo_mnist.pth"))
model.eval()                     # modo evaluación


SimpleCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (full1): Linear(in_features=4608, out_features=128, bias=True)
  (full2): Linear(in_features=128, out_features=10, bias=True)
)

## Using the CIFAR-10 dataset

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),       
    transforms.Normalize((0.5,0.5,0.5), (0.5,)) #this is like the scaler of tensorflow
])


train_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./data", train=True, download=True, transform=transform), #we use
    batch_size=64,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./data", train=False, download=True, transform=transform),
    batch_size=1000,
    shuffle=False
)